PART I

QUESTIONS 
Q1: Some of the facilities charge a fee to members, but some do not.
Write a SQL query to produce a list of the names of the facilities that do. 

In [ ]:
SELECT name
FROM Facilities
WHERE membercost > 0;

Q2: How many facilities do not charge a fee to members? 

In [ ]:
SELECT COUNT(*)
FROM Facilities
WHERE membercost = 0;

Q3: Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question. 

In [ ]:
SELECT facid, name AS facility_name, membercost, monthlymaintenance
FROM Facilities
WHERE membercost > 0
AND membercost < 0.2 * monthlymaintenance;

Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator. 

In [ ]:
SELECT *
FROM Facilities
WHERE facid IN (1, 5);

Q5: Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100. Return the name and monthly maintenance of the facilities
in question. 

In [ ]:
SELECT name AS facility_name,
       CASE WHEN monthlymaintenance > 100 THEN 'expensive'
            ELSE 'cheap' END AS label
FROM Facilities;

Q6: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution. 

In [ ]:
SELECT firstname, surname
FROM Members
WHERE joindate = (SELECT MAX(joindate) FROM Members);

Q7: Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name.

In [ ]:
SELECT DISTINCT CONCAT(firstname, ' ', surname) AS member_name,
       name AS tennis_court
FROM Members
JOIN Bookings ON Members.memid = Bookings.memid
JOIN Facilities ON Bookings.facid = Facilities.facid
WHERE Facilities.name LIKE 'Tennis Court%'
ORDER BY member_name;

Q8: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries. 

In [ ]:
SELECT Facilities.name AS facility_name,
       CONCAT(Members.firstname, ' ', Members.surname) AS member_name,
       CASE WHEN Members.memid = 0 THEN Facilities.guestcost * Bookings.slots
            ELSE Facilities.membercost * Bookings.slots END AS cost
FROM Bookings
JOIN Members ON Bookings.memid = Members.memid
JOIN Facilities ON Bookings.facid = Facilities.facid
WHERE starttime LIKE '2012-09-14%'
AND cost > 30
ORDER BY cost DESC;

Q9: This time, produce the same result as in Q8, but using a subquery. 

In [ ]:
SELECT facility_name, member_name, cost
FROM (
    SELECT Facilities.name AS facility_name,
           CONCAT(Members.firstname, ' ', Members.surname) AS member_name,
           CASE WHEN Members.memid = 0 THEN Facilities.guestcost * Bookings.slots
                ELSE Facilities.membercost * Bookings.slots END AS cost
    FROM Bookings
    JOIN Members ON Bookings.memid = Members.memid
    JOIN Facilities ON Bookings.facid = Facilities.facid
    WHERE starttime LIKE '2012-09-14%'
) AS bookings_cost
WHERE cost > 30
ORDER BY cost DESC;

PART II

In [6]:
import sqlite3
from sqlite3 import Error

 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT *
        FROM FACILITIES
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
(0, 'Tennis Court 1', 5, 25, 10000, 200)
(1, 'Tennis Court 2', 5, 25, 8000, 200)
(2, 'Badminton Court', 0, 15.5, 4000, 50)
(3, 'Table Tennis', 0, 5, 320, 10)
(4, 'Massage Room 1', 9.9, 80, 4000, 3000)
(5, 'Massage Room 2', 9.9, 80, 4000, 3000)
(6, 'Squash Court', 3.5, 17.5, 5000, 80)
(7, 'Snooker Table', 0, 5, 450, 15)
(8, 'Pool Table', 0, 5, 400, 15)


QUESTIONS:

Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!

In [8]:
def select_facilities_with_revenue_less_than_1000(conn):
    """
    Query facilities with a total revenue less than 1000
    :param conn: the Connection object
    :return: rows with facility name and total revenue
    """
    cur = conn.cursor()
    
    query = """
        SELECT 
            name AS facility_name,
            SUM(
                CASE WHEN memid = 0 THEN guestcost * slots
                     ELSE membercost * slots END
            ) AS total_revenue
        FROM Bookings
        JOIN Facilities ON Bookings.facid = Facilities.facid
        GROUP BY name
        HAVING total_revenue < 1000
        ORDER BY total_revenue;
        """
    cur.execute(query)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)

def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("Facilities with revenue less than 1000:")
        select_facilities_with_revenue_less_than_1000(conn)
 
if __name__ == '__main__':
    main()


2.6.0
Facilities with revenue less than 1000:
('Table Tennis', 180)
('Snooker Table', 240)
('Pool Table', 270)


Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

In [9]:
import sqlite3

def create_connection(db_file):
    """Create a database connection to the SQLite database specified by the db_file."""
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)
    return conn

def members_and_recommendations_report(conn):
    """Produce a report of members and who recommended them in alphabetic surname, firstname order."""
    query = """
        SELECT m1.surname || ', ' || m1.firstname AS member_name,
               m2.surname || ', ' || m2.firstname AS recommended_by
        FROM Members AS m1
        LEFT JOIN Members AS m2 ON m1.recommendedby = m2.memid
        ORDER BY m1.surname, m1.firstname;
    """
    cur = conn.cursor()
    cur.execute(query)
    rows = cur.fetchall()
    for row in rows:
        print(row)

def main():
    database = "sqlite_db_pythonsqlite.db"  
    conn = create_connection(database)
    with conn:
        print("Members and Recommendations Report:")
        members_and_recommendations_report(conn)

if __name__ == '__main__':
    main()


Members and Recommendations Report:
('Bader, Florence', 'Stibbons, Ponder')
('Baker, Anne', 'Stibbons, Ponder')
('Baker, Timothy', 'Farrell, Jemima')
('Boothe, Tim', 'Rownam, Tim')
('Butters, Gerald', 'Smith, Darren')
('Coplin, Joan', 'Baker, Timothy')
('Crumpet, Erica', 'Smith, Tracy')
('Dare, Nancy', 'Joplette, Janice')
('Farrell, David', None)
('Farrell, Jemima', None)
('GUEST, GUEST', None)
('Genting, Matthew', 'Butters, Gerald')
('Hunt, John', 'Purview, Millicent')
('Jones, David', 'Joplette, Janice')
('Jones, Douglas', 'Jones, David')
('Joplette, Janice', 'Smith, Darren')
('Mackenzie, Anna', 'Smith, Darren')
('Owen, Charles', 'Smith, Darren')
('Pinker, David', 'Farrell, Jemima')
('Purview, Millicent', 'Smith, Tracy')
('Rownam, Tim', None)
('Rumney, Henrietta', 'Genting, Matthew')
('Sarwin, Ramnaresh', 'Bader, Florence')
('Smith, Darren', None)
('Smith, Darren', None)
('Smith, Jack', 'Smith, Darren')
('Smith, Tracy', None)
('Stibbons, Ponder', 'Tracy, Burton')
('Tracy, Burton', No

Q12: Find the facilities with their usage by member, but not guests

In [10]:
import sqlite3

def create_connection(db_file):
    """Create a database connection to the SQLite database specified by the db_file."""
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)
    return conn

def facilities_usage_by_member_only(conn):
    """Find the facilities with their usage by member, but not guests."""
    query = """
        SELECT f.name AS facility_name,
               COUNT(b.bookid) AS usage_by_member
        FROM Facilities AS f
        JOIN Bookings AS b ON f.facid = b.facid
        WHERE b.memid != 0
        GROUP BY facility_name;
    """
    cur = conn.cursor()
    cur.execute(query)
    rows = cur.fetchall()
    for row in rows:
        print(row)

def main():
    database = "sqlite_db_pythonsqlite.db"  
    conn = create_connection(database)
    with conn:
        print("Facilities usage by member only:")
        facilities_usage_by_member_only(conn)

if __name__ == '__main__':
    main()


Facilities usage by member only:
('Badminton Court', 344)
('Massage Room 1', 421)
('Massage Room 2', 27)
('Pool Table', 783)
('Snooker Table', 421)
('Squash Court', 195)
('Table Tennis', 385)
('Tennis Court 1', 308)
('Tennis Court 2', 276)


Q13: Find the facilities usage by month, but not guests

In [11]:
import sqlite3

def create_connection(db_file):
    """Create a database connection to the SQLite database specified by the db_file."""
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)
    return conn

def facilities_usage_by_month(conn):
    """Find the facilities usage by month, but not guests."""
    query = """
        SELECT f.name AS facility_name,
               strftime('%m', starttime) AS month,
               COUNT(b.bookid) AS usage_count
        FROM Bookings AS b
        JOIN Facilities AS f ON b.facid = f.facid
        WHERE b.memid != 0
        GROUP BY facility_name, month;
    """
    cur = conn.cursor()
    cur.execute(query)
    rows = cur.fetchall()
    for row in rows:
        print(row)

def main():
    database = "sqlite_db_pythonsqlite.db"  
    conn = create_connection(database)
    with conn:
        print("Facilities usage by month (excluding guests):")
        facilities_usage_by_month(conn)

if __name__ == '__main__':
    main()


Facilities usage by month (excluding guests):
('Badminton Court', '07', 51)
('Badminton Court', '08', 132)
('Badminton Court', '09', 161)
('Massage Room 1', '07', 77)
('Massage Room 1', '08', 153)
('Massage Room 1', '09', 191)
('Massage Room 2', '07', 4)
('Massage Room 2', '08', 9)
('Massage Room 2', '09', 14)
('Pool Table', '07', 103)
('Pool Table', '08', 272)
('Pool Table', '09', 408)
('Snooker Table', '07', 68)
('Snooker Table', '08', 154)
('Snooker Table', '09', 199)
('Squash Court', '07', 23)
('Squash Court', '08', 85)
('Squash Court', '09', 87)
('Table Tennis', '07', 48)
('Table Tennis', '08', 143)
('Table Tennis', '09', 194)
('Tennis Court 1', '07', 65)
('Tennis Court 1', '08', 111)
('Tennis Court 1', '09', 132)
('Tennis Court 2', '07', 41)
('Tennis Court 2', '08', 109)
('Tennis Court 2', '09', 126)
